In [1]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model


### Register model

In [2]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [3]:
from federatedscope.contrib.trainer.laplacian_trainer_with_domain_separation_KLD import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "


Device: cuda


 89.1### Register metric

### Create the config

#### Set data, model, trainer and metric

In [4]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("scripts/B-FHTL_exp_scripts/Graph-DT/fed_dom_sep_KLD.yaml")
cfg.data.save_dir = 'test_dir'
#cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
#cfg.model.dropout = 0.5
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
#cfg.trainer.type = 'laplacian_trainer'
#cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']

cfg.params = CN()
cfg.params.alpha=0.1
cfg.params.diff_importance = 0.
cfg.params.csd_importance=1e2
cfg.params.lam = 0.
cfg.params.eps=1e-15
cfg.params.p=0.

#### configure other options

In [5]:
#cfg.use_gpu = True
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'
cfg.federate.method = 'test'  # 'Laplacian_batch_all_clients_csd_1e3_new'
#cfg.federate.local_update_steps = 20000000
#cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 20000

cfg.federate.client_num = 2
cfg.early_stop.patience = 20000
#cfg.train.optimizer.lr = 0.001
#cfg.train.optimizer.weight_decay = 0.0005
cfg.grad.grad_clip = 5.0
#cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.model.dropout = 0.5
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'
cfg.eval.count_flops = False
cfg.model.hidden=64


In [6]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(False)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [7]:
from yacs.config import CfgNode
client_cfg_file = "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_ours_lr_local_steps.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [8]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


2023-01-14 16:38:34,626 (trainer_builder:11)WARNING: No module named 'federatedscope.contrib.optimizer' in `federatedscope.contrib.trainer`, some modules are not available.


In [9]:



setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2023-01-14 16:38:34,634 (utils:129)INFO: the current machine is at 127.0.1.1
2023-01-14 16:38:34,635 (utils:131)INFO: the current dir is /home/michael/Master-Thesis/CKIM_Competition
2023-01-14 16:38:34,635 (utils:132)INFO: the output dir is exp/test_dir/test_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20230114163834


In [10]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_KLD_client import LaplacianDomainSeparation_KLD_Client
from federatedscope.contrib.workers.laplacian_server import LaplacianServer

Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianDomainSeparation_KLD_Client,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

rho: 0.0
server params: 
encoder_atom.atom_embedding_list.0.weight


/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
2023-01-14 16:38:37,672 (fed_runner:249)INFO: Server #0 has been set up ... 
2023-01-14 16:38:37,685 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 2
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: MSELoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
  

encoder_atom.atom_embedding_list.1.weight
encoder_atom.atom_embedding_list.2.weight
encoder_atom.atom_embedding_list.3.weight
encoder_atom.atom_embedding_list.4.weight
encoder_atom.atom_embedding_list.5.weight
encoder_atom.atom_embedding_list.6.weight
encoder_atom.atom_embedding_list.7.weight
encoder_atom.atom_embedding_list.8.weight
encoder_atom.atom_embedding_list.9.weight
encoder_atom.atom_embedding_list.10.weight
encoder_atom.atom_embedding_list.11.weight
encoder_atom.atom_embedding_list.12.weight
encoder_atom.atom_embedding_list.13.weight
encoder_atom.atom_embedding_list.14.weight
encoder_atom.atom_embedding_list.15.weight
encoder_atom.atom_embedding_list.16.weight
encoder_atom.atom_embedding_list.17.weight
encoder_atom.atom_embedding_list.18.weight
encoder_atom.atom_embedding_list.19.weight
encoder_atom.atom_embedding_list.20.weight
encoder_atom.atom_embedding_list.21.weight
encoder.weight
encoder.bias
local_gnn.convs.0.nn.linears.0.weight
local_gnn.convs.0.nn.linears.0.bias
loca

2023-01-14 16:38:43,333 (laplacian_with_domain_separation_KLD_client:140)INFO: {'Role': 'Client #2', 'Round': 0, 'Results_raw': {'train_total': 2560, 'train_loss': 3178.452629, 'train_imp_ratio': -4585.219279, 'train_avg_loss': 1.241583}}


diff loss: 0.0
diff loss: 0.0
round number: 1
diff loss: 0.0
diff loss: 0.0
diff loss: -0.0
diff loss: 0.0
diff loss: -0.0
diff loss: -0.0
diff loss: -0.0
diff loss: -0.0
diff loss: -0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
dif

...

2023-01-14 16:39:18,968 (laplacian_with_domain_separation_KLD_client:140)INFO: {'Role': 'Client #2', 'Round': 1, 'Results_raw': {'train_total': 2560, 'train_loss': 983.880112, 'train_imp_ratio': -1350.295043, 'train_avg_loss': 0.384328}}
2023-01-14 16:39:18,971 (server:480)INFO: {'Role': 'Server #', 'Round': 1, 'Results_avg': {'test_total': 88.5, 'test_loss': 144.2025, 'test_imp_ratio': -530.068151, 'test_avg_loss': 1.333598, 'val_total': 88.0, 'val_loss': 142.035739, 'val_imp_ratio': -610.004873, 'val_avg_loss': 1.325354}}
2023-01-14 16:39:18,971 (monitor:513)INFO: current_best=-10000, should_save=False
2023-01-14 16:39:18,972 (monitor:513)INFO: current_best=-610.004873, should_save=True
2023-01-14 16:39:19,038 (laplacian_server:160)INFO: Server #0: Starting evaluation at the end of round 1.
2023-01-14 16:39:19,040 (laplacian_server:167)INFO: ----------- Starting a new training round (Round #2) -------------


diff loss: 0.0
round number: 1


2023-01-14 16:39:19,164 (client:410)INFO: {'Role': 'Client #1', 'Round': 2, 'Results_raw': {'test_total': 113, 'test_loss': 240.696835, 'test_imp_ratio': -130.451199, 'test_avg_loss': 2.13006, 'val_total': 113, 'val_loss': 162.524177, 'val_imp_ratio': -55.606092, 'val_avg_loss': 1.438267}}
2023-01-14 16:39:19,165 (monitor:513)INFO: current_best=-55.606092, should_save=True
2023-01-14 16:39:19,166 (client:431)INFO: Client: #1, val_imp_ratio: -55.606092. model saved at exp/test_dir/test_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20230114163834/model1.pth
2023-01-14 16:39:19,298 (client:410)INFO: {'Role': 'Client #2', 'Round': 2, 'Results_raw': {'test_total': 64, 'test_loss': 10.616761, 'test_imp_ratio': -525.988223, 'test_avg_loss': 0.165887, 'val_total': 63, 'val_loss': 16.351588, 'val_imp_ratio': -879.430383, 'val_avg_loss': 0.259549}}
2023-01-14 16:39:19,299 (monitor:513)INFO: current_best=-879.430383, should_save=True
2023-01-14 16:39:19,300 (client:431)INFO: Client: #2, val_imp_ratio: -8

round number: 1
round number: 1
round number: 1
round number: 2
diff loss: 0.0
diff loss: 0.0
diff loss: -0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: -0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: -0.0
diff loss: 0.0
diff loss: 0.0
diff loss: -0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: -0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: -0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: 0.0
diff loss: -0.0
diff loss: 0.0

KeyboardInterrupt: 

4

In [19]:
data

{1: {},
 2: {'train': <torch_geometric.loader.dataloader.DataLoader at 0x7f1e1de251c0>,
  'val': <torch_geometric.loader.dataloader.DataLoader at 0x7f1e9d72b040>,
  'test': <torch_geometric.loader.dataloader.DataLoader at 0x7f1ec872d850>,
  'num_label': 0}}

In [ ]:
data

In [ ]:
import os

In [ ]:
os.environ['CUBLAS_WORKSPACE_CONFIG']